In [1]:
import pandas as pd
import time 
import pickle
import os
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

In [2]:
#import pre-processed data from pickle 
#df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_text_wo_names.pickle")
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2011_text_wo_names.pickle")
#df_12= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2012_text_wo_names.pickle")
#df_13= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2013_text_wo_names.pickle")
df_14= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2014_text_wo_names.pickle")
#df_15= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2015_text_wo_names.pickle")
#df_16= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2016_text_wo_names.pickle")
#df_17= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2017_text_wo_names.pickle")
#df_18= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2018_text_wo_names.pickle")
#df_19 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2019_text_wo_names.pickle")
#df_20 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2020_text_wo_names.pickle")
#df_21 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2021_text_wo_names.pickle")
#df_22 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2022_text_wo_names_(1).pickle")
df_14

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
0,"[request, song, write, support, withdraw, sale...",1,0,[Mike],None,Former BBC Radio 1 DJ Mike Read has requested ...,6,2014,0
15,"[label, decline, comment, request, withdraw, s...",1,0,"[Record, Angel]",True,"""Record label Angel Air declined to comment on...",6,2014,0
16,"[spokesman, say, song, obviously, decision, ukip]",1,0,[Mike],None,"A UKIP spokesman said: ""This is Mike's song an...",6,2014,0
17,"[think, shame, treat, harshly, many, right, me...",2,0,[],None,"""We do think it is a shame that he has been tr...",6,2014,0
21,"[seek, donate, money, another, charity, work, ...",1,0,[Ebola],None,"""We will seek to donate all the money to anoth...",6,2014,0
...,...,...,...,...,...,...,...,...,...
3399973,"[site, compile, two, playlist, response, depar...",0,1,[Taylor],None,"""The site has compiled two playlists in respon...",2170631,2014,1
3399977,"[culminate, find, recur, character, female, in...",2,0,[Doctor],None,It culminated with the Doctor finding out that...,2170636,2014,0
3399984,"[hear, claim, immediately, pour, scorn, dismis...",1,0,[],None,"""When the Doctor does hear these claims, he im...",2170636,2014,0
3399985,"[transpire, correct, entire, concept, reveal, ...",1,0,[],None,"""It transpires that he is correct, and the ent...",2170636,2014,0


In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [4]:
#apply function to only get rows with an absolute count 
df_14['col_type'] = df_14.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_14 = df_14[df_14["col_type"].notnull()]

#DOC: number of male and female columns
df_14["col_type"].value_counts()  

0.0    652386
1.0    220630
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [5]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [6]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag' 'saga'], #removed liblinear as it is for small + medium datasets & NOT for sparse data
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    #results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Accuracy: {accuracy:.2f}")
    #print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:1000]
    low_coef = sorted_coef[-1000:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])

    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, best_clf, class_report #df_probs_top

**Coefficient Analysis**

In [7]:
df_probs_14 = logistic_regression_year(df_14, 'col_type', 'pre_processed_sent')
df_probs_14


Execution time: 1086.59 seconds


(           feature      coef
 0         pregnant  8.252975
 1        pregnancy  7.451794
 2          harrier  7.129829
 3          briscoe  7.091350
 4    sterilisation  7.021489
 ..             ...       ...
 995    informatics  3.054526
 996          alina  3.053517
 997       arterton  3.052992
 998          raped  3.052557
 999        lavinia  3.052432
 
 [1000 rows x 2 columns],
          feature      coef
 0        stubble -2.724816
 1        encrypt -2.726553
 2        geraint -2.726952
 3       rochdale -2.727363
 4    balochistan -2.727831
 ..           ...       ...
 995    restraint -5.583731
 996    statesman -5.864737
 997     cardinal -5.943474
 998      handgun -5.967009
 999     prostate -6.824662
 
 [1000 rows x 2 columns],
 LogisticRegression(C=10, class_weight={0: 0.3, 1: 0.7}, random_state=42,
                    solver='newton-cg'),
 '              precision    recall  f1-score   support\n\n         0.0       0.84      0.80      0.82    130536\n         1.0       

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [8]:
#open model performance metrics 
with open('results_14.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification report:\n{report}")

Accuracy: 0.73
Classification report:
              precision    recall  f1-score   support

         0.0       0.84      0.80      0.82    130536
         1.0       0.48      0.55      0.51     44068

    accuracy                           0.73    174604
   macro avg       0.66      0.67      0.66    174604
weighted avg       0.75      0.73      0.74    174604



In [13]:
#create DF of highest coef
highest_coef_14 = pd.DataFrame(df_probs_14[0])
highest_coef_14["coef_type"] = "highest"
highest_coef_14["year"] = 2014

#create DF of lowest lowest coef manipulation 
lowest_coef_14 = pd.DataFrame(df_probs_14[1]) 
lowest_coef_14 = lowest_coef_14.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_14["coef_type"] = "lowest" #coef type
lowest_coef_14["year"] = 2014 #year 
highest_coef_14

,feature,coef,coef_type,year
0,pregnant,8.252975,highest,2014
1,pregnancy,7.451794,highest,2014
2,harrier,7.129829,highest,2014
3,briscoe,7.091350,highest,2014
4,sterilisation,7.021489,highest,2014
...,...,...,...,...
995,informatics,3.054526,highest,2014
996,alina,3.053517,highest,2014
997,arterton,3.052992,highest,2014
998,raped,3.052557,highest,2014


In [14]:
#save DF as pickle file per year 
lowest_coef_14.to_pickle('RESULTS14_coef_low.pickle')
highest_coef_14.to_pickle('RESULTS14_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

In [15]:
def load_pickle_files_low(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("low.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

def load_pickle_files_high(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("high.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

In [16]:
#stack dfs together for low and high coefs 
df_low = load_pickle_files_low(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high = load_pickle_files_high(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high

[           feature      coef coef_type  year
 0         pregnant  3.693514   highest  2012
 1             baby  3.158639   highest  2012
 2           female  3.043889   highest  2012
 3              rap  2.908451   highest  2012
 4          implant  2.901732   highest  2012
 ..             ...       ...       ...   ...
 995          sheet  0.265721   highest  2012
 996      whitehawk  0.265642   highest  2012
 997  heartbreaking  0.265527   highest  2012
 998         rights  0.265453   highest  2012
 999        winning  0.265396   highest  2012
 
 [1000 rows x 4 columns],
             feature      coef coef_type  year
 0          pregnant  3.947211   highest  2013
 1               rap  3.134167   highest  2013
 2         boyfriend  2.970981   highest  2013
 3              baby  2.843502   highest  2013
 4            female  2.827021   highest  2013
 ..              ...       ...       ...   ...
 995    specifically  0.263976   highest  2013
 996     provocation  0.263957   highest  20